In [1]:
!git clone https://github.com/aashishkhimasia/google-research

Cloning into 'google-research'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [2]:
%pip install -r /content/google-research/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmind/dmvr.git to /tmp/pip-req-build-wodrcf63
  Running command git clone -q https://github.com/deepmind/dmvr.git /tmp/pip-req-build-wodrcf63
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-bkknc1fk
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-bkknc1fk
     |████████████████████████████████| 489.6 MB 27 kB/s 
     |████████████████████████████████| 1.1 MB 51.7 MB/s 
     |████████████████████████████████| 5.7 MB 61.8 MB/s 
     |████████████████████████████████| 4.9 MB 49.8 MB/s 
     |████████████████████████████████| 1.3 MB 48.0 MB/s 
     |████████████████████████████████| 1.3 MB 64.5 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 463 kB 60.7 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=51b371153cfbe5

In [ ]:
#import tensorflow as tf

In [ ]:
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#print('Running on TPU ', tpu.cluster_spec().as_dict())

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [9]:
!wget "gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz" 

gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz: Unsupported scheme ‘gs’.


In [14]:
from google.colab import auth 
auth.authenticate_user()

# Download pretrained embeddings and vocabularies from gcs bucket and extract under vatt

!gsutil cp gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz 
!tar -xzf "/content/vision_vatt_misc_data.tgz" -C "/content/google-research/vatt/"

In [19]:
%cd /content/google-research

/content/google-research


Main.py from vatt (with edits from LG)

In [21]:
import pprint

# from absl import app
# from absl import flags
from absl import logging

from vatt.configs import factory as config_factory
from vatt.experiments import finetune
from vatt.experiments import pretrain

# flags.DEFINE_string('task', 'PRETRAIN', 'PRETRAIN or FINETUNE.')
# flags.DEFINE_string('mode', 'train', 'train or eval.')
# flags.DEFINE_string('model_dir', None, 'Default path for the experiment.')
# flags.DEFINE_string('model_arch', 'Tx_FAC', 'Arch of the model.')
# flags.DEFINE_string('override_checkpoint', None,
#                     ('Path to a checkpoint for initialization. '
#                      'If this is passed, the model is initialized from this '
#                      'checkpoint, even if there is a valid latest checkpoint '
#                      'inside the model_dir.'))
# flags.DEFINE_string('config_file', None,
#                     ('Path to a YAML config file containing the dictionary of '
#                      'parameters to override the original params defined '
#                      'under configs/'))
# flags.DEFINE_string('params_override', None,
#                     'A safe_dumped str of a dictionary of parameters')
# flags.DEFINE_string('strategy_type', 'tpu', 'Type of the distribution strategy')
# flags.DEFINE_string('tpu', None, 'Address of the TPU device')


# FLAGS = flags.FLAGS


def get_params():
  """Constructs the configuration of the experiment."""

  task = "FINETUNE"  # FLAGS.task
  model_arch = "Tx_FAC"  # FLAGS.model_arch
  params = config_factory.build_experiment_configs(
      task=task,
      model_arch=model_arch,
      )

  # if FLAGS.config_file:
  #   params.override_from_file(FLAGS.config_file)

 # if FLAGS.params_override:
 #    params.override_from_str(FLAGS.params_override)

  params.override({
      'mode': "eval",  # FLAGS.mode,
      'model_dir': None, # FLAGS.model_dir,
      'checkpoint_path': 'gs://dissertation-adk/checkpoint', #None, # FLAGS.override_checkpoint,
      'strategy_config': {'tpu': None, # FLAGS.tpu,
                          'distribution_strategy': "tpu" # FLAGS.strategy_type
                          },
  })

  return params



def main():
  # del argv  # Unused.

  params = get_params()
  logging.info('Model Parameters: %s', pprint.pformat(params.as_dict()))

  print(str(params.model_dir))
  params.eval.input.name = "toy_ds"
  if params.task.lower() == 'pretrain':
    executor = pretrain.get_executor(params=params)

  elif params.task.lower() == 'finetune':
    executor = finetune.get_executor(params=params)

  else:
    raise ValueError('Task not found: %s.' % params.task)

  return executor.run(mode=params.mode)


if __name__ == '__main__':
  # app.run(main)
  main()

None


UnimplementedError: ignored

In [ ]:
%ls